# torch.optim
---

SGD등의 파라미터 최적화 알고리즘이 구현

학습 단계(loop)에서 최적화는 세단계로 이뤄집니다

1. optimizer.zero_grad()를 호출하여 모델 매개변수의 변화도를 재설정합니다. 기본적으로 변화도는 더해지기(add up) 때문에 중복 계산을 막기 위해 반복할 때마다 명시적으로 0으로 설정합니다.
    - 변화도가 더해지는 이유는 Dezero에서 동일한 변수를 사용하여 덧셈을 하면 제대로 미분을 못하기 때문이다. 일반적으로 더해지게 해서 해결하는 것이다
    #

2. loss.backwards()를 호출하여 예측 손실(prediction loss)을 역전파합니다. PyTorch는 각 매개변수에 대한 손실의 변화도를 저장합니다.


3. 변화도를 계산한 뒤에는 optimizer.step()을 호출하여 역전파 단계에서 수집된 변화도로 매개변수를 조정합니다.

### 1. (들어가기 앞서) 하이퍼파라미터
---
> 하이퍼파라미터(Hyperparameter)는 모델 최적화 과정을 제어할 수 있는 조절 가능한 매개변수입니다.  
> 서로 다른 하이퍼파라미터 값은 모델 학습과 수렴율(convergence rate)에 영향을 미칠 수 있습니다.

- 학습 시에는 다음과 같은 하이퍼파라미터를 정의합니다:

    - 에폭(epoch) 수 - 데이터셋을 반복하는 횟수

    - 배치 크기(batch size) - 매개변수가 갱신되기 전 신경망을 통해 전파된 데이터 샘플의 수

    - 학습률(learning rate) - 각 배치/에폭에서 모델의 매개변수를 조절하는 비율. 값이 작을수록 학습 속도가 느려지고, 값이 크면 학습 중 예측할 수 없는 동작이 발생할 수 있습니다.

### 2. Dezero의 Optimizer
---

In [ ]:
import numpy as np

class Optimizer:
    def __init__(self):
        self.target = None
        self.hooks = []

    def setup(self, target): #매개변수를 갖는 클래스(Model 또는 Layer)를 인스턴스 변수인 target으로 설정합니다
        self.target = target
        return self

    def updata(self):
        #None 이외의 매개변수를 리스트에 모아줌
        params = [p for p in self.target.params() if p.grad is not None]

        for f in self.hooks:
            f(params)

        for param in params:
            self.update_one(param)

    def update_one(self, param): # 구체적인 매개변수 갱신을 위한 함수
        raise NotImplementedError()

    def add_hook(self, f): #전처리를 수행하는 함수
        self.hooks.append(f)

기본적으로 제공하는 Optimizer을 상속받아서 각자의 최적화 기법을 구현한다. 

pytorch의 경우도 마찬가지로 기본 클래스인 torch.optim.Optimizer(params, defaults)을 상속받아서 각자의 최적화 기법을 구현한다.

### 3. pytorch의 다양한 최적화 기법
---
> 자세한 사항 : https://pytorch.org/docs/stable/optim.html

- Adadelta : Adadelta 알고리즘을 구현합니다.

- Adagrad : Adagrad 알고리즘을 구현합니다.

- Adam : Adam 알고리즘을 구현합니다.

- AdamW : AdamW 알고리즘을 구현합니다.

- SparseAdam : 희소 텐서에 적합한 Adam 알고리즘의 지연 버전을 구현합니다.

- Adamax : Adamax 알고리즘(무한대 표준에 기반한 Adam의 변형)을 구현합니다.

- ASGD : 평균 확률적 경사 하강법을 구현합니다.

- LBFGS : minFunc 에서 크게 영감을 받은 L-BFGS 알고리즘을 구현 합니다.

- NAdam : NAdam 알고리즘을 구현합니다.

- RAdam : RAdam 알고리즘을 구현합니다.

- RMSprop : RMSprop 알고리즘을 구현합니다.

- Rprop : 탄력적인 역전파 알고리즘을 구현합니다.

- SGD : 확률적 경사 하강법을 구현합니다(선택적으로 모멘텀 포함).

### 4. 사용법
---
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate) # 아담으로 옵티마이저 설정

### 파이토치 권고 코딩 스타일
---
```py
# 데이터로더로부터 데이터와 정답을 받아옴
for data, label in DataLoader():
    # 모델의 예측값 계산
    prediction = model(data)

    # 손실 함수를 이용해 오차 계산
    loss = LossFunction(prediction, label)

    # 오차 역전파
    loss.backward()

    # 신경망 가중치 수정
    optimizer.step()
```